## Final

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def create_plots(major_folder, llm_folders):

    # Define the CSV files to process
    csv_files = [
        'incorrect_likelihoods_ID.csv', 'incorrect_likelihoods_OOD.csv',
        'ppa_scores_ID.csv', 'ppa_scores_OOD.csv',
        'recall_imbalance_ID.csv', 'recall_imbalance_OOD.csv',
        'error_bins_ID.csv', 'error_bins_OOD.csv'
    ]

    for llm_name, folder_list in llm_folders.items():
        print(f"Creating plots for {llm_name}...")
        mean_values = {}
        for csv_file in csv_files:
            # if 'ID' in csv_file:
            #   continue
            for folder_name in folder_list:
                option_folder = os.path.join(major_folder, folder_name, 'varying_option')
                position_folder = os.path.join(major_folder, folder_name, 'varying_position')
                df_option = pd.read_csv(os.path.join(option_folder, csv_file))
                df_position = pd.read_csv(os.path.join(position_folder, csv_file))
                if 'recall' in csv_file:
                    mean_values_option = df_option.iloc[-2, 1:].values
                    mean_values_position = df_position.iloc[-2, 1:].values
                else:
                  mean_values_option = df_option.iloc[-1, 1:].values
                  mean_values_position = df_position.iloc[-1, 1:].values
                if csv_file+'_option' in mean_values:
                    mean_values[csv_file+'_option'].append(mean_values_option)
                    mean_values[csv_file+'_position'].append(mean_values_position)
                else:
                    mean_values[csv_file+'_option'] = []
                    mean_values[csv_file+'_option'].append(mean_values_option)
                    mean_values[csv_file+'_position'] = []
                    mean_values[csv_file+'_position'].append(mean_values_position)
        for csv_file in csv_files:
            # if 'ID' in csv_file:
            #     continue
            mean_values_option = np.array(mean_values[csv_file+'_option'])
            mean_values_position = np.array(mean_values[csv_file+'_position'])
            y_label = '_'.join(csv_file.split('_')[:2])
            title = '_'.join(csv_file.split(' ')[:2])
            if 'error' not in csv_file:
              plt.figure(figsize=(12, 6))
              N = mean_values_option.shape[0]
              num_elements = mean_values_option.shape[1]
              ind = np.arange(N)
              width = 0.2
              fig, ax = plt.subplots()
              # Define colors
              colors = plt.cm.tab10(np.linspace(0, 1, 4))  # Adjust color map for more subtle colors
              colors[0] = np.array([76, 146, 195, 255])/255
              colors[1] = np.array([255, 152, 62, 255])/255
              colors[2] = np.array([86, 179, 86, 255])/255
              colors[3] = np.array([222, 83, 83, 255])/255
              x_labels = ['A', 'B', 'C', 'D']
              y_label = '_'.join(csv_file.split('_')[:2])
              for i in range(num_elements):
                  ax.bar(ind + i*width, mean_values_option[:,i], width, label=x_labels[i], color=colors[i])
              # ax.set_xlabel('Model')
              ax.set_ylabel(y_label)
              ax.set_title('Bar charts for all values separated by elements', pad=40)
              ax.set_xticks(ind + width)
              ax.set_xticklabels(folder_list, rotation=45)
              ax.legend()
              plot_file = os.path.join(major_folder, f'{llm_name}_{csv_file[:-4]}_comparison_plot.png')
              plt.savefig(plot_file)
              plt.close()
              print(f"    Plot saved as '{plot_file}'")

              plt.figure(figsize=(12, 6))
              N = mean_values_position.shape[0]
              num_elements = mean_values_position.shape[1]
              ind = np.arange(N)
              width = 0.2
              fig, ax = plt.subplots()
              # Define colors
              colors = plt.cm.tab10(np.linspace(0, 1, 4))  # Adjust color map for more subtle colors
              colors[0] = np.array([76, 146, 195, 255])/255
              colors[1] = np.array([255, 152, 62, 255])/255
              colors[2] = np.array([86, 179, 86, 255])/255
              colors[3] = np.array([222, 83, 83, 255])/255
              x_labels = ['0', '1', '2', '3']
              y_label = '_'.join(csv_file.split('_')[:2])
              for i in range(num_elements):
                  ax.bar(ind + i*width, mean_values_position[:,i], width, label=x_labels[i], color=colors[i])
              # ax.set_xlabel('Model')
              ax.set_ylabel(y_label)
              ax.set_title(f'{title} comparison for {llm_name} before and after tuning', pad=40)
              ax.set_xticks(ind + width)
              ax.set_xticklabels(folder_list, rotation=45)
              ax.legend()
              plot_file = os.path.join(major_folder, f'{llm_name}_{csv_file[:-4]}_comparison_plot.png')
              plt.savefig(plot_file)
              plt.close()
              print(f"    Plot saved as '{plot_file}'")
            else:
              plt.figure(figsize=(12, 6))
              data_T = mean_values_option.T
              N = mean_values_option.shape[0]
              ind = np.arange(N)
              fig, ax = plt.subplots()
              bottom = np.zeros(N)
              # Define colors
              colors = plt.cm.tab10(np.linspace(0, 1, 4))  # Adjust color map for more subtle colors
              colors[0] = np.array([76, 146, 195, 255])/255
              colors[1] = np.array([255, 152, 62, 255])/255
              colors[2] = np.array([86, 179, 86, 255])/255
              colors[3] = np.array([222, 83, 83, 255])/255
              x_labels = ['A', 'B', 'C', 'D']
              for i in range(data_T.shape[0]):
                  ax.bar(ind, data_T[i], bottom=bottom, color=colors[i], label=x_labels[i])
                  bottom += data_T[i].astype(float)
              ax.set_xlabel('Model')
              ax.set_ylabel(y_label)
              ax.set_title(f'{title} comparison for {llm_name} before and after tuning')
              ax.set_xticks(ind)
              ax.set_xticklabels(folder_list, rotation=45)
              ax.legend()
              plot_file = os.path.join(major_folder, f'{llm_name}_{csv_file[:-4]}_comparison_plot.png')
              plt.savefig(plot_file)
              plt.close()
              print(f"    Plot saved as '{plot_file}'")

              plt.figure(figsize=(12, 6))
              data_T = mean_values_position.T
              N = mean_values_position.shape[0]
              ind = np.arange(N)
              fig, ax = plt.subplots()
              bottom = np.zeros(N)
              # Define colors
              colors = plt.cm.tab10(np.linspace(0, 1, 4))  # Adjust color map for more subtle colors
              colors[0] = np.array([76, 146, 195, 255])/255
              colors[1] = np.array([255, 152, 62, 255])/255
              colors[2] = np.array([86, 179, 86, 255])/255
              colors[3] = np.array([222, 83, 83, 255])/255
              x_labels = ['0', '1', '2', '3']
              for i in range(data_T.shape[0]):
                  ax.bar(ind, data_T[i], bottom=bottom, color=colors[i], label=x_labels[i])
                  bottom += data_T[i].astype(float)
              ax.set_xlabel('Model')
              ax.set_ylabel(y_label)
              ax.set_title(f'{title} comparison for {llm_name} before and after tuning')
              ax.set_xticks(ind)
              ax.set_xticklabels(folder_list, rotation=45)
              ax.legend()
              plot_file = os.path.join(major_folder, f'{llm_name}_{csv_file[:-4]}_comparison_plot.png')
              plt.savefig(plot_file)
              plt.close()
              print(f"    Plot saved as '{plot_file}'")

# Define the major folder path where all the LLM folders are present
major_folder = '/content/drive/MyDrive/685 Project/mmlu 03'  # Update with the path to your major folder

# Define the LLM names and associated folder lists
llm_folders = {
    # 'gemma-2b-it': ['gemma-2b-it', 'gemma-2b-it-lora-tuned', 'gemma-2b-it-lora-tuned-per-8k', 'gemma-2b-it-lora-tuned-per-16k'],
    'gemma-2b-it': ['gemma-2b-it', 'gemma-2b-it-lora-tuned', 'gemma-2b-it-lora-tuned-per-8k', 'gemma-2b-it-lora-tuned-per-16k'],
    'llama-2-7b': ['llama-2-7b', 'llama-2-7b-lora-tuned', 'llama-2-7b-lora-tuned-per-8k', 'llama-2-7b-lora-tuned-per-16k']
    # Add more LLMs as needed
}

# Call the function to create plots
create_plots(major_folder, llm_folders)

Creating plots for gemma-2b-it...
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/gemma-2b-it_incorrect_likelihoods_ID_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/gemma-2b-it_incorrect_likelihoods_ID_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/gemma-2b-it_incorrect_likelihoods_OOD_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/gemma-2b-it_incorrect_likelihoods_OOD_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/gemma-2b-it_ppa_scores_ID_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/gemma-2b-it_ppa_scores_ID_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/gemma-2b-it_ppa_scores_OOD_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/gemma-2b-it_ppa_scores_OOD_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 

<ipython-input-3-b1242a1ff3c1>:81: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_incorrect_likelihoods_OOD_comparison_plot.png'


<ipython-input-3-b1242a1ff3c1>:49: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12, 6))


    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_ppa_scores_ID_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_ppa_scores_ID_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_ppa_scores_OOD_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_ppa_scores_OOD_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_recall_imbalance_ID_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_recall_imbalance_ID_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_recall_imbalance_OOD_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_recall_imbalance_OOD_comparison_plot.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_error_bins_ID_comparison_plot.png'


...

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Example data
# data = np.array([
#     [1, 2, 3, 4],
#     [5, 6, 7, 8],
#     [9, 10, 11, 12]
# ])

# data_T = data.T

# N = data.shape[0]

# ind = np.arange(N)

# fig, ax = plt.subplots()

# bottom = np.zeros(N)

# colors = ['r', 'g', 'b', 'y']

# for i in range(data_T.shape[0]):
#     ax.bar(ind, data_T[i], bottom=bottom, color=colors[i], label=f'Value {i+1}')
#     bottom += data_T[i]

# ax.set_xlabel('Elements')
# ax.set_ylabel('Values')
# ax.set_title('Stacked Bar Chart for Each Element')
# ax.set_xticks(ind)
# ax.set_xticklabels([f'Element {i+1}' for i in range(N)])
# ax.legend()

# # Display the plot
# plt.show()


In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def create_plots(major_folder, llm_folders):

    # Define the CSV files to process
    csv_files = [
        'incorrect_likelihoods_ID.csv', 'incorrect_likelihoods_OOD.csv',
        'ppa_scores_ID.csv', 'ppa_scores_OOD.csv',
        'recall_imbalance_ID.csv', 'recall_imbalance_OOD.csv',
        'error_bins_ID.csv', 'error_bins_OOD.csv'
    ]

    for llm_name, folder_list in llm_folders.items():
        print(f"Creating plots for {llm_name}...")
        mean_values = {}
        for csv_file in csv_files:
            # if 'ID' in csv_file:
            #   continue
            for folder_name in folder_list:
                option_folder = os.path.join(major_folder, folder_name, 'varying_option')
                position_folder = os.path.join(major_folder, folder_name, 'varying_position')
                df_option = pd.read_csv(os.path.join(option_folder, csv_file))
                df_position = pd.read_csv(os.path.join(position_folder, csv_file))
                if 'recall' in csv_file:
                    mean_values_option = df_option.iloc[-2, 1:].values
                    mean_values_position = df_position.iloc[-2, 1:].values
                else:
                    mean_values_option = df_option.iloc[-1, 1:].values
                    mean_values_position = df_position.iloc[-1, 1:].values
                if csv_file+'_option' in mean_values:
                    mean_values[csv_file+'_option'].append(mean_values_option)
                    mean_values[csv_file+'_position'].append(mean_values_position)
                else:
                    mean_values[csv_file+'_option'] = []
                    mean_values[csv_file+'_option'].append(mean_values_option)
                    mean_values[csv_file+'_position'] = []
                    mean_values[csv_file+'_position'].append(mean_values_position)
        for csv_file in csv_files:
            # if 'ID' in csv_file:
            #     continue
            mean_values_option = np.array(mean_values[csv_file+'_option'])
            mean_values_position = np.array(mean_values[csv_file+'_position'])
            y_label = '_'.join(csv_file.split('_')[:2])
            title = '_'.join(csv_file.split(' ')[:2])
            if 'error' not in csv_file:
                plt.figure(figsize=(12, 6))
                N = mean_values_option.shape[0]
                num_elements = mean_values_option.shape[1]
                ind = np.arange(N)
                width = 0.2
                fig, ax = plt.subplots()
                # Define colors
                colors = plt.cm.tab10(np.linspace(0, 1, 4))  # Adjust color map for more subtle colors
                colors[0] = np.array([76, 146, 195, 255])/255
                colors[1] = np.array([255, 152, 62, 255])/255
                colors[2] = np.array([86, 179, 86, 255])/255
                colors[3] = np.array([222, 83, 83, 255])/255
                x_labels = ['A', 'B', 'C', 'D']
                y_label = '_'.join(csv_file.split('_')[:2])
                for i in range(num_elements):
                    ax.bar(ind + i*width, mean_values_option[:, i], width, label=x_labels[i], color=colors[i])
                # ax.set_xlabel('Model')
                ax.set_ylabel(y_label)
                ax.set_title(f'{y_label} comparison for {llm_name} before & after tuning', pad=40)
                ax.set_xticks(ind + width)
                ax.set_xticklabels(["base_model", "lora_tuned", "lora_tuned_per_8k", "lora_tuned_per_16k", "lora_tuned_per_32k"], rotation=45, ha='right')
                ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=num_elements)
                fig.tight_layout()
                fig.subplots_adjust(top=0.85)
                plot_file = os.path.join(major_folder, f'{llm_name}_{csv_file[:-4]}_comparison_plot_option.png')
                plt.savefig(plot_file)
                plt.close()
                print(f"    Plot saved as '{plot_file}'")

                plt.figure(figsize=(12, 6))
                N = mean_values_position.shape[0]
                num_elements = mean_values_position.shape[1]
                ind = np.arange(N)
                width = 0.2
                fig, ax = plt.subplots()
                # Define colors
                colors = plt.cm.tab10(np.linspace(0, 1, 4))  # Adjust color map for more subtle colors
                colors[0] = np.array([76, 146, 195, 255])/255
                colors[1] = np.array([255, 152, 62, 255])/255
                colors[2] = np.array([86, 179, 86, 255])/255
                colors[3] = np.array([222, 83, 83, 255])/255
                x_labels = ['0', '1', '2', '3']
                y_label = '_'.join(csv_file.split('_')[:2])
                for i in range(num_elements):
                    ax.bar(ind + i*width, mean_values_position[:, i], width, label=x_labels[i], color=colors[i])
                # ax.set_xlabel('Model')
                ax.set_ylabel(y_label)
                ax.set_title(f'{y_label} comparison for {llm_name} before & after tuning', pad=40)
                ax.set_xticks(ind + width)
                ax.set_xticklabels(["base_model", "lora_tuned", "lora_tuned_per_8k", "lora_tuned_per_16k", "lora_tuned_per_32k"], rotation=45, ha='right')
                ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=num_elements)
                fig.tight_layout()
                fig.subplots_adjust(top=0.85)
                plot_file = os.path.join(major_folder, f'{llm_name}_{csv_file[:-4]}_comparison_plot_position.png')
                plt.savefig(plot_file)
                plt.close()
                print(f"    Plot saved as '{plot_file}'")
            else:
                plt.figure(figsize=(12, 6))
                data_T = mean_values_option.T
                N = mean_values_option.shape[0]
                ind = np.arange(N)
                fig, ax = plt.subplots()
                bottom = np.zeros(N)
                # Define colors
                colors = plt.cm.tab10(np.linspace(0, 1, 4))  # Adjust color map for more subtle colors
                colors[0] = np.array([76, 146, 195, 255])/255
                colors[1] = np.array([255, 152, 62, 255])/255
                colors[2] = np.array([86, 179, 86, 255])/255
                colors[3] = np.array([222, 83, 83, 255])/255
                x_labels = ['top_1_bin', 'top_2_bin', 'top_3_bin', 'top_4_bin']
                for i in range(data_T.shape[0]):
                    ax.bar(ind, data_T[i], bottom=bottom, color=colors[i], label=x_labels[i])
                    bottom += data_T[i].astype(float)
                # ax.set_xlabel('Model')
                ax.set_ylabel('Bin #')
                ax.set_title(f'Gold Answer Bin % for {llm_name} before & after tuning', pad=40)
                ax.set_xticks(ind)
                ax.set_xticklabels(["base_model", "lora_tuned", "lora_tuned_per_8k", "lora_tuned_per_16k", "lora_tuned_per_32k"], rotation=45, ha='right')
                ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=data_T.shape[0])
                fig.tight_layout()
                fig.subplots_adjust(top=0.85)
                plot_file = os.path.join(major_folder, f'{llm_name}_{csv_file[:-4]}_comparison_plot_option.png')
                plt.savefig(plot_file)
                plt.close()
                print(f"    Plot saved as '{plot_file}'")

                plt.figure(figsize=(12, 6))
                data_T = mean_values_position.T
                N = mean_values_position.shape[0]
                ind = np.arange(N)
                fig, ax = plt.subplots()
                bottom = np.zeros(N)
                # Define colors
                colors = plt.cm.tab10(np.linspace(0, 1, 4))  # Adjust color map for more subtle colors
                colors[0] = np.array([76, 146, 195, 255])/255
                colors[1] = np.array([255, 152, 62, 255])/255
                colors[2] = np.array([86, 179, 86, 255])/255
                colors[3] = np.array([222, 83, 83, 255])/255
                x_labels = ['top_1_bin', 'top_2_bin', 'top_3_bin', 'top_4_bin']
                for i in range(data_T.shape[0]):
                    ax.bar(ind, data_T[i], bottom=bottom, color=colors[i], label=x_labels[i])
                    bottom += data_T[i].astype(float)
                # ax.set_xlabel('Model')
                ax.set_ylabel("Bin #")
                ax.set_title(f'Gold Answer Bin % for {llm_name} before & after tuning', pad=40)
                ax.set_xticks(ind)
                ax.set_xticklabels(["base_model", "lora_tuned", "lora_tuned_per_8k", "lora_tuned_per_16k", "lora_tuned_per_16k"], rotation=45, ha='right')
                ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=data_T.shape[0])
                fig.tight_layout()
                fig.subplots_adjust(top=0.85)
                plot_file = os.path.join(major_folder, f'{llm_name}_{csv_file[:-4]}_comparison_plot_position.png')
                plt.savefig(plot_file)
                plt.close()
                print(f"    Plot saved as '{plot_file}'")

# Define the major folder path where all the LLM folders are present
major_folder = '/content/drive/MyDrive/685 Project/mmlu 03'  # Update with the path to your major folder

# Define the LLM names and associated folder lists
llm_folders = {
    # 'gemma-2b-it': ['gemma-2b-it', 'gemma-2b-it-lora-tuned', 'gemma-2b-it-lora-tuned-per-8k', 'gemma-2b-it-lora-tuned-per-16k'],
    'llama-2-7b': ['llama-2-7b', 'llama-2-7b-lora-tuned', 'llama-2-7b-lora-tuned-per-8k', 'llama-2-7b-lora-tuned-per-16k', 'llama-2-7b-lora-tuned-per-32k']
    # Add more LLMs as needed
}

# Call the function to create plots
create_plots(major_folder, llm_folders)

Creating plots for llama-2-7b...
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_incorrect_likelihoods_ID_comparison_plot_option.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_incorrect_likelihoods_ID_comparison_plot_position.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_incorrect_likelihoods_OOD_comparison_plot_option.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_incorrect_likelihoods_OOD_comparison_plot_position.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_ppa_scores_ID_comparison_plot_option.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_ppa_scores_ID_comparison_plot_position.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_ppa_scores_OOD_comparison_plot_option.png'
    Plot saved as '/content/drive/MyDrive/685 Project/mmlu 03/llama-2-7b_ppa_scores_OOD_comparison_plot_posit

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

...

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>